# Unofficial ACI Guide

## Python 3 - Get APIC Users Example

List local users on the APIC controller.  
No Cisco ACI-related toolkits, SDKs, or bindings were harmed in the creation of this example.

In [1]:
import requests
import json
import pandas as pd
import os
import getpass

Enter your controller hostname/IP and user credentials in config.py.  
If there is no config.py, we prompt for entries.  
The getpass() function allows for shadow entries.  

In [2]:
if os.path.isfile('./config.py'):
    from config import controller, username, password

try:
    controller
except NameError:
    controller = input("Controller Hostname/IP: ")

try:
    username
except NameError:
    username = input("APIC Username: ")
    
try: 
    password 
except NameError:
    password = getpass.getpass("APIC Password: ")


base_url = "https://" + str(controller) + "/api/"
auth_bit = "aaaLogin.json"
auth_url = base_url + auth_bit

In [3]:
# JSON auth data 
auth_data = {
  "aaaUser":{
    "attributes":{
      "name":username,
      "pwd":password
    }
  }
}

## Construct the Request

In [4]:
# We use verify=False to allow self-signed certs. 
# The disable_warnings() line prevents warning messages.
requests.packages.urllib3.disable_warnings() 
s = requests.session()
s.post(auth_url, json=auth_data, verify=False)

<Response [200]>

In [5]:
# Set up the URLs to query.
qstring = "?rsp-subtree=children"
# Construct the user post URL
user_class="node/mo/uni/userext.json"
user_url = base_url + user_class + qstring
user_url

'https://10.18.188.101/api/node/mo/uni/userext.json?rsp-subtree=children'

In [6]:
# Initiate the request and get JSON.
users = s.get(user_url, verify=False)
users = users.json()

In [7]:
# Print as pretty as a dandelion
print(json.dumps(users, indent=4, sort_keys=True))

{
    "imdata": [
        {
            "aaaUserEp": {
                "attributes": {
                    "annotation": "",
                    "childAction": "",
                    "descr": "",
                    "dn": "uni/userext",
                    "extMngdBy": "",
                    "lcOwn": "local",
                    "modTs": "2018-11-17T13:52:59.962-05:00",
                    "monPolDn": "uni/fabric/monfab-default",
                    "name": "",
                    "nameAlias": "",
                    "newSubscr": "yes",
                    "ownerKey": "",
                    "ownerTag": "",
                    "pwdStrengthCheck": "yes",
                    "status": "",
                    "uid": "0"
                },
                "children": [
                    {
                        "aaaRsToUserEp": {
                            "attributes": {
                                "childAction": "deleteNonPresent",
                                "forceResolve"

In [8]:
#Grab the DN
dn = users['imdata'][0]['aaaUserEp']['attributes']['dn']

In [9]:
# Set up empty lists
name_list = []
firstname_list = []
lastname_list = []
status_list = []
phone_list = []
rn_list = []

In [10]:
# Grab the user objects.
users_list = users['imdata'][0]['aaaUserEp']['children']
#users_list

# in each object in "users_list", not all are users.
# we test if the class name for the object is "aaaUser"
# if so, we then grab the attributes key and read the value.
# since the attributes value is a dictionary, we ask for the relevent key
# and assign that value to the variable.
# For example, the object class is aaaUser, there is { attributes: {some_dict} }
# and some_dict has a list of key/values like rn:blah, name:blah, etc.

for user in users_list:
    if "aaaUser" in user: 
        #print(user)
        rn = user['aaaUser']['attributes']['rn']
        name = user['aaaUser']['attributes']['name']  
        firstname = user['aaaUser']['attributes']['firstName']
        lastname = user['aaaUser']['attributes']['lastName']
        status = user['aaaUser']['attributes']['accountStatus']
        phone = user['aaaUser']['attributes']['phone']
        name_list.append(name)
        rn_list.append(rn)
        firstname_list.append(firstname)
        lastname_list.append(lastname)
        phone_list.append(phone)
        status_list.append(status)

In [11]:
zlist = zip(status_list, name_list, firstname_list, lastname_list, phone_list, rn_list)

In [12]:
df_input = list(zlist)
df = pd.DataFrame(df_input, columns=("Status","User Name","First Name","Last Name","Phone","RN"))

In [13]:
df.head()

,Status,User Name,First Name,Last Name,Phone,RN
0,active,api-stcorry,,,,user-api-stcorry
1,active,api,Example,User,256-512-1024,user-api
2,active,stcorry,,,,user-stcorry
3,active,joddavis,,,,user-joddavis
4,active,eptracker,,,,user-eptracker
